In [1]:
from sklearn.model_selection import train_test_split
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import *
import torch,torchvision
from tqdm import tqdm
device = 'cuda'
PROJECT_NAME = 'Rock-Paper-Scissors-Clf'

In [2]:
def load_data():
    labels = {}
    labels_r = {}
    data = []
    idx = -1
    for folder in tqdm(os.listdir('./data/')):
        idx += 1
        labels[idx] = folder
        labels_r[folder] = idx
        for file in os.listdir(f'./data/{folder}'):
            img = f'./data/{folder}/{file}'
            img = cv2.imread(img)
            img = cv2.resize(img,(112,112))
            data.append([img/255.0,idx])
    np.random.shuffle(data)
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(d[1])
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=False)
    return X_train,X_test,y_train,y_test,X,y,data,idx,labels,labels_r

In [ ]:
X_train,X_test,y_train,y_test,X,y,data,idx,labels,labels_r = load_data()

  0%|                                                     | 0/3 [00:00<?, ?it/s]

In [ ]:
X_train = torch.from_numpy(np.array(X_train)).view(-1,3,112,112).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).view(-1,3,112,112).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device)
y_test = torch.from_numpy(np.array(y_test)).to(device)

In [ ]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [ ]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.argmax(pred))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [ ]:
def get_pred(model):
    model.eval()
    for file in os.listdir('./test_data/'):
        img = cv2.imread(f'./test_data/{file}')
        img = cv2.resize(img,(112,112))
        img = torch.from_numpy(np.array(img)).to(device).view(-1,3,112,112).float()
        pred = model(img)
        pred = torch.argmax(pred)
        plt.figure(figsize=(12,6))
        plt.imshow(img.view(112,112,3).cpu().detach().numpy())
        plt.title(f'{labels[pred]}')
        plt.savefig(f'./preds/{file}')
        plt.close()
    model.train()

In [ ]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.max_pool2d = MaxPool2d((2,2),(2,2))
        self.activation = ReLU()
        self.conv1 = Conv2d(3,6,(2,2))
        self.conv1batchnorm = BatchNorm2d(6)
        self.conv2 = Conv2d(6,9,(3,3))
        self.conv2batchnorm = BatchNorm2d(9)
        self.conv3 = Conv2d(9,12,(2,2))
        self.conv3batchnorm = BatchNorm2d(12)
        self.conv4 = Conv2d(12,15,(3,3))
        self.conv4batchnorm = BatchNorm2d(15)
        self.linear1 = Linear(15*5*5,256)
        self.linear1batchnorm = BatchNorm1d(256)
        self.linear2 = Linear(256,512)
        self.linear2batchnorm = BatchNorm1d(512)
        self.linear3 = Linear(512,1024)
        self.linear3batchnorm = BatchNorm1d(1024)
        self.linear4 = Linear(1024,512)
        self.linear4batchnorm = BatchNorm1d(512)
        self.output = Linear(512,3)
    
    def forward(self,X):
        preds = self.max_pool2d(self.activation(self.conv1batchnorm(self.conv1(X))))
        preds = self.max_pool2d(self.activation(self.conv2batchnorm(self.conv2(preds))))
        preds = self.max_pool2d(self.activation(self.conv3batchnorm(self.conv3(preds))))
        preds = self.max_pool2d(self.activation(self.conv4batchnorm(self.conv4(preds))))
        preds = preds.view(-1,15*5*5)
        preds =self.activation(self.linear1batchnorm(self.linear1(preds)))
        preds =self.activation(self.linear2batchnorm(self.linear2(preds)))
        preds =self.activation(self.linear3batchnorm(self.linear3(preds)))
        preds =self.activation(self.linear4batchnorm(self.linear4(preds)))
        preds = self.output(preds)
        return preds

In [ ]:
model = Model().to(device)

In [ ]:
criterion = CrossEntropyLoss()

In [ ]:
from torch.optim import *

In [ ]:
optimizer = Adam(model.parameters(),lr=0.001)

In [ ]:
epochs = 100

In [ ]:
batch_size = 32

In [ ]:
import wandb

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,112,112).to(device).float()
        y_batch = y_train[idx:idx+batch_size].to(device).float()
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
    get_pred(model)
    for file in os.listdir('./preds/'):
        wandb.log({f'Img/{file}':wandb.Image(cv2.imread(f'./preds/{file}'))})
wandb.finish()